In [ ]:
%load_ext autoreload
%autoreload 2
import sympy as sym
from sympy import pi
from sympy import init_printing
import numpy as np
import matplotlib.pyplot as plt
import os
import yaml
from contextlib import redirect_stdout
from braininversion.BiotStokesConvergence import (compute_temporal_mm,generate_expression,
                                                  compute_forcing_terms,check_interface_conditions,
                                                  boundary_id, compute_temporal_convergence, names,
                                                  compute_order)
T = 1
num_time_steps = [4,8,16,32,64, 128]
spatial_resolution = 5

material_parameter = dict()
E = 1
nu = 0.4999

u_degree = 2
p_degree = 1

material_parameter["c"] = 1
material_parameter["kappa"] = 1
material_parameter["lambda"] =  10# nu*E/((1.0-2.0*nu)*(1.0+nu))
material_parameter["mu_s"] =   5# E/(2.0*(1.0+nu))
material_parameter["rho_s"] = 1
material_parameter["mu_f"] = 1
material_parameter["rho_f"] = 1
material_parameter["alpha"] = 1.0
material_parameter["gamma"] = 1.0
material_parameter["lmbda"] = material_parameter["lambda"]

x,y,z, t = sym.symbols("x, y, z, t")

interf_x = 0.0

u = sym.Matrix([1,1])*sym.sin(2*pi*t)
d, p_p, p_f, phi = compute_temporal_mm(u, p_p = sym.sin(3*pi*t))

check_interface_conditions(u, p_f, d, p_p, interf_x)
f_fluid, f_porous, g_source = compute_forcing_terms(u, p_f, d, p_p)

expressions = {"pF":p_f, "pP":p_p, "d":d, "u":u, "phi":phi,
              "f_fluid":f_fluid, "f_porous":f_porous, "g_source":g_source}

for name, exp in expressions.items():
    with open(f'../results/MMS/temporal_expressions/{name}.tex', 'w') as f:
        with redirect_stdout(f):
            print(sym.latex(exp))

In [ ]:
L2_error, num_results_temp, delta_ts = compute_temporal_convergence(num_time_steps, T,
                                            u, p_f, d, p_p, phi, material_parameter,
                                            u_degree, p_degree, spatial_resolution)
num_steps = num_time_steps[-1]
dt = T/num_steps
times = np.linspace(0, T, num_steps + 1)

In [ ]:
L2_rates = {}
for n in names:
    L2_rates[n] = compute_order(L2_error[n], delta_ts).tolist()

In [ ]:
plt.figure(figsize=(7,5))
for n in names:
    plt.loglog(1/delta_ts, L2_error[n], "*-", label=n)
plt.loglog(1/delta_ts, 5e-1*delta_ts , "-", label="$O(h^1)$")

ticks = [10,20,40,80]
plt.legend()
plt.grid(which="both")
plt.xlabel("$ 1 / \Delta t $")
plt.ylabel("L2 error")
plt.xticks( ticks, ticks)
plt.tight_layout()
plt.savefig("../results/MMS/temporal_conv.pdf")

In [ ]:
conv_results = {"L2rates":L2_rates,
                "T":T,
                "L2error":L2_error,
                "spatial_resolution":spatial_resolution,
                "num_time_steps":spatial_resolution,
                "material_parameter":material_parameter}


with open('../results/MMS/temporal_conv.yml', 'w') as f:
    yaml.dump(conv_results, f)